In [1]:
from langgraph.graph import StateGraph,START,END
from langchain_google_genai import ChatGoogleGenerativeAI
from typing import TypedDict
from dotenv import load_dotenv

In [4]:
model=ChatGoogleGenerativeAI(model="gemini-2.0-flash")

In [2]:
load_dotenv()

True

In [6]:
class Prompt_chaining(TypedDict):
    topic:str
    outline:str
    blog:str

In [8]:
def gen_outline(state:Prompt_chaining)->Prompt_chaining:
    title=state['topic']
    prompt=f"generate an detailed outline of the given topic,{title}"
    result=model.invoke(prompt)
    state['outline']=result.content
    return state

In [9]:
def gen_blog(state:Prompt_chaining)->Prompt_chaining:
    title=state['topic']
    outline=state['outline']
    prompt=f"generate a good blog on this title-{title} using this outling-{outline}"
    result=model.invoke(prompt)
    state['blog']=result.content
    return state

In [10]:
graph=StateGraph(Prompt_chaining)
graph.add_node("gen_outline",gen_outline)
graph.add_node("gen_blog",gen_blog)
graph.add_edge(START,"gen_outline")
graph.add_edge("gen_outline","gen_blog")
graph.add_edge("gen_blog",END)
wf=graph.compile()

In [12]:
final_state=wf.invoke({'topic':'Human'})

In [13]:
final_state['outline']

'## Detailed Outline: The Human Being\n\nThis outline explores the multifaceted nature of humans, encompassing biological, psychological, social, and cultural aspects.\n\n**I. Biological Aspects: The Human Body & Its Function**\n\n   A. **Anatomy and Physiology:**\n      1. **Skeletal System:**\n         a. Structure and Function (Support, Movement, Protection, Blood Cell Production)\n         b. Bone Types and Composition\n         c. Joints and Ligaments\n         d. Common Skeletal Disorders\n      2. **Muscular System:**\n         a. Types of Muscles (Skeletal, Smooth, Cardiac)\n         b. Muscle Contraction and Movement\n         c. Muscle Fatigue and Adaptation\n         d. Common Muscular Disorders\n      3. **Nervous System:**\n         a. Central Nervous System (Brain and Spinal Cord):\n            i. Brain Structure and Function (Cerebrum, Cerebellum, Brainstem)\n            ii. Spinal Cord Function (Relay Station, Reflexes)\n         b. Peripheral Nervous System:\n         